In [1]:
import requests # library to handle requests
import lxml.html as lh
import bs4 as bs
import urllib.request

import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis

In [2]:
url   = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

In [3]:
def scrape_table_bs4(cname,cols):
    page  = urllib.request.urlopen(url).read()
    soup  = bs.BeautifulSoup(page,'lxml')
    table = soup.find("table",class_=cname)
    header = [head.findAll(text=True)[0].strip() for head in table.find_all("th")]
    data   = [[td.findAll(text=True)[0].strip() for td in tr.find_all("td")]
              for tr in table.find_all("tr")]
    data    = [row for row in data if len(row) == cols]
    raw_df = pd.DataFrame(data,columns=header)
    return raw_df

def scrape_table_lxml(XPATH,cols):
    page = requests.get(url)
    doc = lh.fromstring(page.content)
    table_content = doc.xpath(XPATH)
    for table in table_content:
        headers = [th.text_content().strip() for th in table.xpath('//th')]
        headers = headers[0:3]
        data    = [[td.text_content().strip() for td in tr.xpath('td')] 
                   for tr in table.xpath('//tbody/tr')]
        data    = [row for row in data if len(row) == cols]
        raw_df = pd.DataFrame(data,columns=headers)
        return raw_df

In [4]:

raw_TorontoPostalCodes = scrape_table_bs4("wikitable",3)


print("# Toronto Postal codes stored in data")
print(raw_TorontoPostalCodes.info(verbose=True))


# Toronto Postal codes stored in data
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 180 entries, 0 to 179
Data columns (total 3 columns):
Postal Code     180 non-null object
Borough         180 non-null object
Neighborhood    180 non-null object
dtypes: object(3)
memory usage: 4.3+ KB
None


In [5]:
# -----------------------------------------------------
# Only process the cells that have an assigned borough. 
# Ignore cells with a borough that is Not assigned.
# -----------------------------------------------------
TorontoPostalCodes=raw_TorontoPostalCodes[~raw_TorontoPostalCodes['Borough'].isin(['Not assigned'])]

# Sort and Reset index.
TorontoPostalCodes=TorontoPostalCodes.sort_values(by=['Postal Code','Borough','Neighborhood'], ascending=[1,1,1]).reset_index(drop=True)

# -----------------------------------------------------
# If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.
# For example the value of the Borough and the Neighborhood columns will be Queen's Park.
# -----------------------------------------------------
TorontoPostalCodes.loc[TorontoPostalCodes['Neighborhood'] == 'Not assigned', ['Neighborhood']] = TorontoPostalCodes['Borough']
check_unassigned_post_state_sample = TorontoPostalCodes.loc[TorontoPostalCodes['Borough'] == 'Queen\'s Park']
#print('DEBUG:',check_unassigned_post_state_sample) ; # Print sample borough problem post state

# -----------------------------------------------------
# More than one neighborhood can exist in one postal code area. 
# For example, in the table on the Wikipedia page, you will notice that M5A is listed twice 
# and has two neighborhoods: Harbourfront and Regent Park. 
# These two rows will be combined into one row with the neighborhoods separated with a comma.
# -----------------------------------------------------
TorontoPostalCodes = TorontoPostalCodes.groupby(['Postal Code','Borough'])['Neighborhood'].apply(', '.join).reset_index()


In [6]:
# defining dataframe to utilise Pandas
df = TorontoPostalCodes

In [7]:
# print shape but only rows
print(df.shape[0])

103


In [8]:
print(df)

    Postal Code           Borough  \
0           M1B       Scarborough   
1           M1C       Scarborough   
2           M1E       Scarborough   
3           M1G       Scarborough   
4           M1H       Scarborough   
5           M1J       Scarborough   
6           M1K       Scarborough   
7           M1L       Scarborough   
8           M1M       Scarborough   
9           M1N       Scarborough   
10          M1P       Scarborough   
11          M1R       Scarborough   
12          M1S       Scarborough   
13          M1T       Scarborough   
14          M1V       Scarborough   
15          M1W       Scarborough   
16          M1X       Scarborough   
17          M2H        North York   
18          M2J        North York   
19          M2K        North York   
20          M2L        North York   
21          M2M        North York   
22          M2N        North York   
23          M2P        North York   
24          M2R        North York   
25          M3A        North York   
2

In [9]:
geo_df=pd.read_csv('http://cocl.us/Geospatial_data')


In [10]:
geo_df.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [11]:
df.head()

,Postal Code,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [12]:
df_join = pd.merge(df, geo_df)

In [15]:
display(df_join)

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park",43.727929,-79.262029
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848


In [13]:
# provides a description of the data set stating how many entries there are, the mean (meaning what is the most common address - excluding outliers), and other statistical categories
df_join.describe()

,Latitude,Longitude
count,103.000000,103.000000
mean,43.704608,-79.397153
std,0.052463,0.097146
min,43.602414,-79.615819
25%,43.660567,-79.464763
50%,43.696948,-79.388790
75%,43.745320,-79.340923
max,43.836125,-79.160497


In [30]:
# Creating a formatted dataframe by Borough
Analysis = df_join.groupby('Borough')

In [33]:
# Provides a count and other characteristics.
Analysis['Borough', 'Latitude', 'Longitude'].describe()
#  Indicates clustering within the areas and how 'big' the postal code areas are

Latitude                                             \
                    count       mean       std        min        25%   
Borough                                                                
Central Toronto       9.0  43.701980  0.017146  43.672710  43.689574   
Downtown Toronto     19.0  43.654597  0.011709  43.628947  43.647688   
East Toronto          5.0  43.669436  0.008566  43.659526  43.662744   
East York             5.0  43.700303  0.009846  43.685347  43.695344   
Etobicoke            12.0  43.660043  0.041459  43.602414  43.634404   
Mississauga           1.0  43.636966       NaN  43.636966  43.636966   
North York           24.0  43.750727  0.025895  43.709577  43.727847   
Scarborough          17.0  43.766229  0.039272  43.692657  43.744734   
West Toronto          6.0  43.652653  0.011266  43.636847  43.648185   
York                  5.0  43.690797  0.012050  43.673185  43.689026   

                                                  Longitude             \
                        50%        75%        max     count       mean   
Borough                                                                  
Central Toronto   43.704324  43.712751  43.728020       9.0 -79.398954   
Downtown Toronto  43.653206  43.662499  43.679563      19.0 -79.383972   
East Toronto      43.668999  43.676357  43.679557       5.0 -79.324654   
East York         43.705369  43.706397  43.709060       5.0 -79.335851   
Etobicoke         43.652298  43.690759  43.739416      12.0 -79.542074   
Mississauga       43.636966  43.636966  43.636966       1.0 -79.615819   
North York        43.753008  43.768515  43.803762      24.0 -79.429338   
Scarborough       43.770992  43.794200  43.836125      17.0 -79.249085   
West Toronto      43.650265  43.659099  43.669005       6.0 -79.449290   
York              43.691116  43.693781  43.706876       5.0 -79.472633   

                                                                        \
                       std        min        25%        50%        75%   
Borough                                                                  
Central Toronto   0.011728 -79.416936 -79.405678 -79.400049 -79.388790   
Downtown Toronto  0.013615 -79.422564 -79.388438 -79.381752 -79.376474   
East Toronto      0.022994 -79.352188 -79.340923 -79.321558 -79.315572   
East York         0.021944 -79.363452 -79.349372 -79.338106 -79.318389   
Etobicoke         0.032805 -79.594054 -79.560344 -79.537863 -79.517485   
Mississauga            NaN -79.615819 -79.615819 -79.615819 -79.615819   
North York        0.070014 -79.565963 -79.487965 -79.431005 -79.371898   
Scarborough       0.044312 -79.318389 -79.284577 -79.262029 -79.216917   
West Toronto      0.024051 -79.484450 -79.462654 -79.449292 -79.431708   
York              0.034064 -79.518188 -79.487262 -79.476013 -79.453512   

                             
                        max  
Borough                      
Central Toronto  -79.383160  
Downtown Toronto -79.360636  
East Toronto     -79.293031  
East York        -79.309937  
Etobicoke        -79.498509  
Mississauga      -79.615819  
North York       -79.315572  
Scarborough      -79.160497  
West Toronto     -79.419750  
York             -79.428191

In [34]:
# Provides a count and other characteristics.
Analysis['Borough'].describe()
#  North York(1st), Downtown Toronto (2nd) and Scarborough (3rd)

,count,unique,top,freq
Borough,,,,
Central Toronto,9,1,Central Toronto,9
Downtown Toronto,19,1,Downtown Toronto,19
East Toronto,5,1,East Toronto,5
East York,5,1,East York,5
Etobicoke,12,1,Etobicoke,12
Mississauga,1,1,Mississauga,1
North York,24,1,North York,24
Scarborough,17,1,Scarborough,17
West Toronto,6,1,West Toronto,6
